In [1]:
%load_ext jupyter_black

In [151]:
from typing import Iterable
import pandas as pd
import numpy as np
from numpy.typing import NDArray, ArrayLike
import matplotlib.pyplot as plt

# =============================================================================
# TODO: these functions can be moved into a separate module
def sort_unique(x: ArrayLike) -> NDArray[np.float_]:
    return np.sort(np.unique(x))


def normalize(x: NDArray[np.number], keepdims: bool = True) -> NDArray[np.float_]:
    return (x - x.min(keepdims=keepdims)) / (x.max(keepdims=keepdims) - x.min(keepdims=keepdims))


def scale(x: NDArray[np.number], rate: float = 1.0) -> NDArray[np.float_]:
    return normalize(x) * rate + 1
# =============================================================================
P0 = 1013.25 # - mbar
P1 = 25.0 # - mbar
STP = sort_unique([P0, *range(1000, 25 - 1, -25), P1]).astype(np.float_)[::-1]
ERA5_GRID_RESOLUTION = 30.0  # km / px
RATE = ERA5_GRID_RESOLUTION / 2
URMA_GRID_RESOLUTION = 2.5  # km / px

MESOSCALE_BETA = 200.0  # km

def mesoscale(rate: float = 1.0) -> NDArray[np.float_]:
    return scale(np.log(STP), rate=rate)[::-1]


class Mesoscale:
    def __init__(
        self,
        dx: float = 200.0,
        dy: float | None = None,
        km_px: float = URMA_GRID_RESOLUTION,
        *,
        pressure: list[float],
        rate: float = 1.0,
    ) -> None:
        self.p = p = sort_unique(pressure)[::-1]
        self._scale = scale = mesoscale(rate=rate)[np.newaxis, np.isin(STP, p)]
        self.dx, self.dy = scale * np.array([[dx], [dy or dx]])

    def to_pandas(self) -> pd.DataFrame:
        return pd.DataFrame(
            {
                "dx": self.dx,
                "dy": self.dy,
                "px": self.dx / URMA_GRID_RESOLUTION,
                "py": self.dy / URMA_GRID_RESOLUTION,
            },
            index=pd.Index(self.p, name="hPa"),
        ).sort_index()


Mesoscale(
    200, 175, pressure=[P0, 925.0, 850.0, 700.0, 500.0, 300.0], rate=RATE
).to_pandas()  # .loc[[925, 850, 700, 500, 300], :].sort_index()

,dx,dy,px,py
hPa,,,,
300.00,2956.206151,2586.680382,1182.482460,1034.672153
500.00,2704.867923,2366.759433,1081.947169,946.703773
700.00,2338.595685,2046.271224,935.438274,818.508490
850.00,1885.103487,1649.465551,754.041395,659.786220
925.00,1504.229710,1316.200996,601.691884,526.480398
1013.25,200.000000,175.000000,80.000000,70.000000
